In [1]:
import os
import cv2
import tensorflow as tf
import numpy as np
import json

In [2]:
from utilities import *

In [3]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
#LABEL_MAP_NAME = 'mscoco_label_map.pbtxt'
LABEL_MAP_NAME = 'mscoco_label_map.json'

CWD_PATH = os.getcwd()
MODELS_PATH = os.path.abspath(os.path.join(CWD_PATH, 'models'))
LABELS_PATH = os.path.abspath(os.path.join(CWD_PATH, 'labels'))
DATA_PATH = os.path.abspath(os.path.join(CWD_PATH, 'data'))

VIDEO_FILE = 'cows.mp4'
VIDEO_PATH = os.path.join(DATA_PATH, VIDEO_FILE)

MODEL_PATH = os.path.join(MODELS_PATH, MODEL_NAME, 'frozen_inference_graph.pb')
LABEL_MAP_PATH = os.path.join(LABELS_PATH, LABEL_MAP_NAME)

THRESHOLD = 0

In [7]:
with open(LABEL_MAP_PATH) as json_file:
    labels = json.load(json_file)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\fabian.chan\\PycharmProjects\\explore-tf\\labels\\mscoco_label_map.json'

In [10]:
print(MODEL_PATH)
print(LABEL_MAP_PATH)

C:\Users\fabian.chan\PycharmProjects\explore-tf\models\ssd_mobilenet_v1_coco_2017_11_17\frozen_inference_graph.pb
C:\Users\fabian.chan\PycharmProjects\explore-tf\labels\mscoco_label_map.pbtxt


In [11]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(MODEL_PATH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [19]:
 def detect_objects(image_np, sess, detection_graph, threshold=0):
        # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

        # Each box represents a part of the image where a particular object was detected.
        boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

        # Each score represent level of confidence for each of the objects.
        scores = detection_graph.get_tensor_by_name('detection_scores:0')
        classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        # Actual detection.
        (boxes, scores, classes, num_detections) = sess.run(
            [boxes, scores, classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})
        return (boxes, scores, classes, num_detections)
        # filter for target classes within threshold
        boxes = np.squeeze(boxes)
        classes = np.squeeze(classes)
        scores = np.squeeze(scores)
        ix_scores = scores >= threshold
        ix = ix_classes * ix_scores
        tf_data = [boxes[ix], np.extract(ix, classes), np.extract(ix, scores)]

        return tf_data

In [30]:
# WEBCAM
#video_stream = VideoStream(src=0).start() 
# VIDEO File
video_stream = VideoStream(src=VIDEO_PATH).start()
while video_stream.grabbed:
    frame, counter = video_stream.read()
    
    # do some zoo model detection on the frame
#     with detection_graph.as_default():
#         with tf.Session(graph=detection_graph) as sess:
#             data = detect_objects(frame, sess, detection_graph)
#     print(data)
    height = frame.shape[0]
    width = frame.shape[1]
    
    for item in data[0][0]:
        cv2.rectangle(
            frame,
            (int(width * item[1]), int(height * item[0])), 
            (int(width * item[3]), int(height * item[2])), 
            (0, 255, 0),
            1
        )
        
    
    cv2.imshow('frame', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_stream.stop()
cv2.destroyAllWindows()

In [26]:
frame.shape

(1080, 1920, 3)

In [29]:
data[0][0]

array([[0.3248309 , 0.49664423, 0.59975356, 0.59468937],
       [0.38548434, 0.66678685, 0.66046584, 0.8367489 ],
       [0.34657553, 0.39204016, 0.61214197, 0.52060586],
       [0.34657553, 0.39204016, 0.61214197, 0.52060586],
       [0.35224354, 0.4276129 , 0.6171329 , 0.5463513 ],
       [0.41182894, 0.2962485 , 0.55053806, 0.38318932],
       [0.3248309 , 0.49664423, 0.59975356, 0.59468937],
       [0.38618585, 0.67003745, 0.6627948 , 0.8367935 ],
       [0.46798843, 0.8990146 , 0.59496385, 0.9878262 ],
       [0.34657553, 0.39204016, 0.61214197, 0.52060586],
       [0.41182894, 0.2962485 , 0.55053806, 0.38318932],
       [0.3301196 , 0.4282054 , 0.6032401 , 0.5512619 ],
       [0.41182894, 0.2962485 , 0.55053806, 0.38318932],
       [0.38618585, 0.67003745, 0.6627948 , 0.8367935 ],
       [0.47218603, 0.91978544, 0.59303766, 0.98388654],
       [0.3248309 , 0.49664423, 0.59975356, 0.59468937],
       [0.4457125 , 0.2781114 , 0.5457335 , 0.32546836],
       [0.3748945 , 0.65633357,